## **Regression Model** - Solar Flare from RHESSI Mission
* January  09°, 2025
#### ESCOM - IPN

#### *B.S. in Data Science* 
> Sanchez Garcia Miguel Alexander

#### **0° Introduction**

**a.** Get the **libraries** we are going to use

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool

**b.** Import the data

In [38]:
data = pd.read_csv('hessi_flare_list_cleaned.csv')

#### **1° EDA (Exploratory Data Analysis)**

##### *General Data*

**a.** Let´s check the **dimensions** of the dataset

In [39]:
print('Rows: ' + str(data.shape[0]))
print('Columns: ' + str(data.shape[1]))

Rows: 15738
Columns: 14


**b.** Check the first **10 rows**

In [40]:
data.head(10)

,Flare,Date,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,Energy keV,X Pos asec,Y Pos asec,Radial asec,AR,Flags
0,2021229,2002-02-12,02:15:24,02:19:22,02:25:48,624.0,46.0,75270.0,25-50,958,-118,965.0,9809.0,A1 P1
1,2021230,2002-02-12,02:49:04,02:49:38,02:50:48,104.0,20.0,6528.0,6-12,637,-100,645.0,0.0,A1 EE P1 Q1
2,2021213,2002-02-12,21:30:08,21:33:46,21:41:16,668.0,147.0,184773.0,12-25,597,-370,703.0,9811.0,A1 P1
3,2021332,2002-02-13,00:53:28,00:54:54,00:56:52,204.0,16.0,12070.0,12-25,-309,375,486.0,0.0,A1 P1
4,2021356,2002-02-13,02:48:56,02:49:34,02:50:36,100.0,13.0,4995.0,12-25,-385,299,488.0,9825.0,A1 P1
5,2021357,2002-02-13,02:51:56,02:52:30,02:53:00,64.0,65.0,3054.0,6-12,56,56,80.0,0.0,A1 P1
6,2021308,2002-02-13,04:22:24,04:23:58,04:28:04,340.0,22.0,21883.0,12-25,-286,371,468.0,0.0,A1 P1
7,2021312,2002-02-13,08:53:20,08:55:18,09:05:12,712.0,99.0,137741.0,25-50,-388,281,479.0,9825.0,A1 P1
8,2021313,2002-02-13,12:29:32,12:30:58,12:33:20,228.0,29.0,18203.0,12-25,-904,-379,981.0,0.0,A1 P1
9,2021423,2002-02-14,22:22:32,22:25:26,22:26:00,208.0,449.0,137924.0,12-25,-81,315,326.0,9825.0,A0 EE P1 Q1


#### **2° Preparing Data**

**a.** Drop innesesary dimensions

In [41]:
# Drop innesesary columns
data = data.drop(['Flare', 'Energy keV', 'Flags'], axis=1)

In [42]:
data

,Date,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,X Pos asec,Y Pos asec,Radial asec,AR
0,2002-02-12,02:15:24,02:19:22,02:25:48,624.0,46.0,75270.0,958,-118,965.0,9809.0
1,2002-02-12,02:49:04,02:49:38,02:50:48,104.0,20.0,6528.0,637,-100,645.0,0.0
2,2002-02-12,21:30:08,21:33:46,21:41:16,668.0,147.0,184773.0,597,-370,703.0,9811.0
3,2002-02-13,00:53:28,00:54:54,00:56:52,204.0,16.0,12070.0,-309,375,486.0,0.0
4,2002-02-13,02:48:56,02:49:34,02:50:36,100.0,13.0,4995.0,-385,299,488.0,9825.0
...,...,...,...,...,...,...,...,...,...,...,...
15733,2011-12-24,01:46:24,01:46:54,01:47:24,60.0,18.0,5915.0,-913,-345,976.0,1386.0
15734,2011-12-26,15:44:56,15:45:06,15:45:16,20.0,36.0,3881.0,-917,-363,986.0,0.0
15735,2011-12-27,15:37:00,15:37:26,15:37:32,32.0,21.0,2961.0,-918,-378,993.0,1388.0
15736,2011-12-31,05:33:20,05:34:02,05:34:52,92.0,19.0,8061.0,-674,-399,783.0,1389.0


**b.** Set to datetime some dimension

**c.** Get into a list the name of categorical features

In [43]:
# Get categorical features
categorical_features = [col for col in data.columns if data[col].dtype == 'object']

**d.** Split the dataset

In [44]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [45]:
date_columns = ['Date'] 
time_columns = ['Start', 'Peak', 'End']  

# Convert date columns to numerical format
for col in date_columns:
    train_data[col] = pd.to_datetime(train_data[col])
    train_data[col] = (train_data[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
    test_data[col] = pd.to_datetime(test_data[col])
    test_data[col] = (test_data[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')

# Convert time columns to numerical format (seconds since midnight)
for col in time_columns:
    train_data[col] = pd.to_datetime(train_data[col], format='%H:%M:%S').dt.hour * 3600 + \
                      pd.to_datetime(train_data[col], format='%H:%M:%S').dt.minute * 60 + \
                      pd.to_datetime(train_data[col], format='%H:%M:%S').dt.second
    test_data[col] = pd.to_datetime(test_data[col], format='%H:%M:%S').dt.hour * 3600 + \
                        pd.to_datetime(test_data[col], format='%H:%M:%S').dt.minute * 60 + \
                        pd.to_datetime(test_data[col], format='%H:%M:%S').dt.second


In [46]:
train_data

,Date,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,X Pos asec,Y Pos asec,Radial asec,AR
949,11819,23000,23158,23252,252.0,90.0,51369.0,835,-276,879.0,9934.0
14141,15185,70620,70810,70852,232.0,93.0,70198.0,-718,262,765.0,1263.0
10712,13455,77812,77850,77904,92.0,30.0,13118.0,-127,-179,220.0,921.0
2991,11988,84044,84362,85056,1012.0,1276.0,2098145.0,-969,150,980.0,0.0
5125,12154,32156,32210,32264,108.0,13.0,6801.0,571,177,598.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...
5191,12161,83184,83446,84576,1392.0,608.0,1101792.0,-896,-185,916.0,337.0
13418,15077,82292,82598,83184,892.0,891.0,1744854.0,-892,281,936.0,1193.0
5390,12198,45044,45142,45232,188.0,34.0,22667.0,26,-79,83.0,365.0
860,11817,43508,43510,43624,116.0,45.0,22967.0,637,-428,768.0,9932.0


#### **4° Traning the Model**

**a.** Set the model

In [83]:
models = {}
columns = train_data.columns
columns = columns.drop('Date')
for target in columns:
    print(f"Training model for {target}...")
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.1,
        depth=6,
        loss_function='RMSE',
        verbose=50
    )
    x = train_data['Date'].values.reshape(-1, 1)
    y = train_data[target]
    model.fit(x, y)
    models[target] = model

Training model for Start...
0:	learn: 26112.2754345	total: 2.48ms	remaining: 2.48s
50:	learn: 25898.3222587	total: 49ms	remaining: 912ms
100:	learn: 25777.1076496	total: 82.9ms	remaining: 738ms
150:	learn: 25643.8725302	total: 118ms	remaining: 664ms
200:	learn: 25547.6905206	total: 151ms	remaining: 602ms
250:	learn: 25483.0453301	total: 186ms	remaining: 556ms
300:	learn: 25435.7711386	total: 225ms	remaining: 522ms
350:	learn: 25404.2529587	total: 261ms	remaining: 483ms
400:	learn: 25382.5764323	total: 299ms	remaining: 446ms
450:	learn: 25366.2957980	total: 334ms	remaining: 407ms
500:	learn: 25355.3001638	total: 367ms	remaining: 365ms
550:	learn: 25346.9647424	total: 401ms	remaining: 327ms
600:	learn: 25341.3114675	total: 435ms	remaining: 289ms
650:	learn: 25335.6438147	total: 471ms	remaining: 253ms
700:	learn: 25331.7186646	total: 507ms	remaining: 216ms
750:	learn: 25328.0705391	total: 541ms	remaining: 179ms
800:	learn: 25325.5270372	total: 574ms	remaining: 142ms
850:	learn: 25323.3536

In [ ]:
models = {}
for target in train_data.columns:
    print(f"Training model for {target}...")
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.1,
        depth=6,
        loss_function='RMSE',
        verbose=50
    )
    x = train_data.drop(columns=[target])
    y = train_data[target]
    model.fit(x, y)
    models[target] = model

Training model for Start...


KeyError: 0

In [84]:
predictions = {}
for target, model in models.items():
    predictions[target] = model.predict(test_data['Date'].values.reshape(-1, 1))

# Combine predictions into a DataFrame
predictions_df = pd.DataFrame(predictions)

In [85]:
predictions_df

,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,X Pos asec,Y Pos asec,Radial asec,AR
0,40407.256499,39571.437182,39688.376340,282.590114,81.872628,9.235941e+04,-207.974384,-54.440117,820.689273,811.395997
1,39427.343426,39562.652611,40036.188966,476.290688,90.940480,2.175669e+05,20.058369,-28.049411,699.949216,5504.291573
2,36394.296384,36416.759603,36577.790424,273.742705,157.429391,1.578327e+05,716.527062,210.199598,782.232038,1229.427393
3,34214.652525,34319.622591,32682.261638,339.130808,124.984099,2.123415e+05,92.225681,-153.483527,764.234799,835.026225
4,42903.925034,41798.733898,41709.267020,322.988905,164.675804,1.630283e+05,-146.725683,-126.823972,761.913598,846.501814
...,...,...,...,...,...,...,...,...,...,...
3143,40407.256499,39571.437182,39688.376340,282.590114,81.872628,9.235941e+04,-207.974384,-54.440117,820.689273,811.395997
3144,43623.289245,43956.330532,44033.019127,494.541481,897.889826,1.081832e+06,486.737136,281.141661,816.829175,151.100456
3145,32916.486038,32889.219054,33110.617345,347.672235,374.337234,4.462998e+05,84.864504,-80.842497,706.734189,509.102653
3146,53297.688515,53510.734322,51768.596251,343.944457,1068.434010,1.457097e+06,497.937708,-217.469181,581.219842,61.138034


In [89]:
total_mse = mean_squared_error(test_data.drop('Date', axis=1), predictions_df)

print(f"Total MSE: {total_mse}")

total_r2 = r2_score(test_data.drop('Date', axis=1), predictions_df)

print(f"Total R^2: {total_r2}")

Total MSE: 2220532863333.3145
Total R^2: 0.16816842350546302


In [68]:
def reset_datime_format(data):
    date_columns = ['Date'] 
    time_columns = ['Start', 'Peak', 'End']

    # Convert date columns back to datetime format
    for col in date_columns:
        data[col] = pd.to_datetime(data[col], unit='D', origin="1970-01-01")

    # Convert time columns back to HH:MM:SS format
    for col in time_columns:
        data[col] = pd.to_timedelta(data[col], unit='s')
        data[col] = (pd.Timestamp("1970-01-01") + data[col]).dt.time

    # Extract only the date part
    data['Date'] = data['Date'].dt.date

    # Round the time to the nearest second
    data['Start'] = data['Start'].apply(lambda x: x.replace(microsecond=0))
    data['Peak'] = data['Peak'].apply(lambda x: x.replace(microsecond=0))
    data['End'] = data['End'].apply(lambda x: x.replace(microsecond=0))

    return data

In [69]:
reset_datime_format(predictions_df)

,Date,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,X Pos asec,Y Pos asec,Radial asec,AR
0,2006-11-10,03:39:32,03:34:16,03:39:46,306.877150,58.979892,3.726567e+04,554.150102,-145.523399,979.073813,833.818164
1,2002-05-20,00:39:07,00:32:48,00:35:34,89.933974,24.494637,8.832093e+04,635.081153,-276.274544,752.730768,9427.572850
2,2011-07-19,20:07:40,20:07:55,20:09:01,123.731170,34.270427,4.394718e+03,813.749375,220.590815,805.899623,1235.915629
3,2006-06-24,09:46:52,09:42:53,09:51:04,146.531748,65.367947,1.988619e+04,384.937974,-117.628638,826.485660,898.697291
4,2010-07-05,23:02:08,23:02:22,23:01:29,72.136543,19.678450,3.408101e+04,-119.094953,-325.070040,654.032885,825.264406
...,...,...,...,...,...,...,...,...,...,...,...
3143,2006-11-03,09:48:58,09:50:57,09:51:27,130.632454,23.109175,-2.240381e+04,-51.881115,-127.684908,971.484952,454.215603
3144,2002-11-19,10:59:36,10:59:55,11:10:31,996.834043,28.745924,1.648073e+05,570.706403,410.558047,875.930097,208.517017
3145,2003-12-15,22:25:56,22:27:31,22:28:52,232.034925,34.332369,-4.370616e+03,1070.928129,-52.081298,998.869112,529.858721
3146,2005-02-04,02:59:33,03:05:03,03:15:34,1205.041237,2984.598016,1.265220e+06,254.964932,-293.322582,285.064712,-56.796746


In [71]:
def clean_data(data):
    # Get colums (no date and time columns)
    cols = data.columns
    cols = cols.drop(['Date', 'Start', 'Peak', 'End'])

    # Round the predictions to the nearest integer
    data[cols] = data[cols].round().astype(int)

    return data

In [72]:
clean_data(predictions_df)

,Date,Start,Peak,End,Dur_s,Peak_c/s,Total Counts,X Pos asec,Y Pos asec,Radial asec,AR
0,2006-11-10,03:39:32,03:34:16,03:39:46,307,59,37266,554,-146,979,834
1,2002-05-20,00:39:07,00:32:48,00:35:34,90,24,88321,635,-276,753,9428
2,2011-07-19,20:07:40,20:07:55,20:09:01,124,34,4395,814,221,806,1236
3,2006-06-24,09:46:52,09:42:53,09:51:04,147,65,19886,385,-118,826,899
4,2010-07-05,23:02:08,23:02:22,23:01:29,72,20,34081,-119,-325,654,825
...,...,...,...,...,...,...,...,...,...,...,...
3143,2006-11-03,09:48:58,09:50:57,09:51:27,131,23,-22404,-52,-128,971,454
3144,2002-11-19,10:59:36,10:59:55,11:10:31,997,29,164807,571,411,876,209
3145,2003-12-15,22:25:56,22:27:31,22:28:52,232,34,-4371,1071,-52,999,530
3146,2005-02-04,02:59:33,03:05:03,03:15:34,1205,2985,1265220,255,-293,285,-57


In [75]:
january_2022_data = pd.DataFrame({
    'Date': pd.date_range(start='2022-01-01', end='2022-01-31', freq='D')
})


january_2022_predictions = {}
for target, model in models.items():
    january_2022_predictions[target] = model.predict(january_2022_data)

# Combine predictions into a DataFrame
january_2022_predictions_df = pd.DataFrame(january_2022_predictions)

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 0 should be feature with name Start (found Date).